In [1]:
import tensorflow as tf

In [3]:
tf.__version__

'2.8.0'

In [4]:
#ses sinyallerinin özniteliklerini çıkarmak için splitAudioData.pynb dosyasında librosa kütüphanesini kullanmıştık
#burada rastegele 10 tane ses verisi alacağız ve bunları CNN algoritması kullanarak test edeceğiz

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import os

import librosa
import numpy as np

from tqdm import tqdm

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam

In [13]:
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

In [28]:
#verilerimizi import ediyoruz
data = pd.read_csv('../audioData.csv')

In [29]:
data

,feature,class
0,[-6.6391638e+02 7.4633250e+00 -6.2418396e+01 ...,beeBird
1,[-5.16368103e+02 2.06674080e+01 -1.56551895e+...,beeBird
2,[-469.1672 9.647738 -137.6997 -1...,beeBird
3,[-6.3586462e+02 1.5384795e+01 -1.6060805e+02 ...,beeBird
4,[-5.0093127e+02 8.0044546e+00 -1.5133850e+02 ...,beeBird
...,...,...
2148,[-3.32472534e+02 1.16614426e+02 4.51772957e+...,pigeon
2149,[-262.14792 83.66872 14.975486 5...,pigeon
2150,[-284.36514 132.5125 42.36633 5...,pigeon
2151,[-282.64023 99.18566 29.451946 5...,pigeon


In [126]:
x=np.array(data['feature'].tolist())
y=np.array(data['class'].tolist())

In [133]:
x.shape

(2153,)

In [95]:
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(Y))

In [97]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [119]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=0)

In [120]:
num_labels=9

In [121]:
model=Sequential()
model.add(Dense(125, input_shape=(40, )))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#2. gizli
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#3. gizli
model.add(Dense(125))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [122]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 125)               5125      
                                                                 
 activation_28 (Activation)  (None, 125)               0         
                                                                 
 dropout_21 (Dropout)        (None, 125)               0         
                                                                 
 dense_29 (Dense)            (None, 250)               31500     
                                                                 
 activation_29 (Activation)  (None, 250)               0         
                                                                 
 dropout_22 (Dropout)        (None, 250)               0         
                                                                 
 dense_30 (Dense)            (None, 125)              

In [123]:
model.compile(loss='categorical_crossentropy', metrics=["accuracy"], optimizer="adam")

In [124]:
epochscount=300
num_batch_size=32

In [125]:
model.fit(xtrain, ytrain, batch_size=num_batch_size, epochs=epochscount, validation_data=(xtest, ytest), verbose=1)

Epoch 1/300


ValueError: in user code:

    File "c:\python39\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\python39\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\python39\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\python39\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "c:\python39\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\python39\lib\site-packages\keras\engine\input_spec.py", line 228, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" '

    ValueError: Exception encountered when calling layer